# Cassava Leaf Disease Classification (Model)

**Things I've tried**

- Image size (160,160,3) with EfficientNetB4, learning rate 0.001 accuracy got stuck in local minimum of 0.6148.
- Image size (512,512,3) with EfficientNetB4, learning rate 0.001 accuracy got stuck in local minimum of 0.6148.
- Image size (512,512,3) with EfficientNetB4, learning rate 0.0002 accuracy got stuck in local minimum of 0.6137.
- Image size (512,512,3) with EfficientNetB4, learning rate 0.0002, label smoothing 0.1, accuracy got stuck in local minimum of 0.6137.

In [88]:
import json
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
# learning rate decay scheduler
from tensorflow.keras.experimental import CosineDecay

from tensorflow.keras.applications import efficientnet

In [4]:
tf.__version__

'2.5.0-dev20201213'

In [92]:
#TPU Configurations
AUTO = tf.data.experimental.AUTOTUNE

# other configurations
class configurations():
    def __init__(self, IMG_SIZE, EPOCHS, BATCH_SIZE, DROPOUT_RATE, INIT_LEARNING_RATE, ALPHA):
        '''
        :param IMG_SIZE: a tuple of length 2 that defines the height and width of the image
        :param EPOCHS: no. of epochs to train the neural network
        :param BATCH_SIZE: size of a batch of images
        :param DROPOUT_RATE: the ratio of nodes to remove randomly during training
        :param INIT_LEARNING_RATE: initial learning rate for gradient descent
        :param ALPHA: parameter required for CosineDecay learning rate scheduler
        '''
        self.IMG_SIZE = IMG_SIZE
        self.INPUT_SHAPE = IMG_SIZE + (3,)
        self.EPOCHS = EPOCHS
        self.BATCH_SIZE = BATCH_SIZE
        self.DROPOUT_RATE = DROPOUT_RATE
        self.INIT_LEARNING_RATE = INIT_LEARNING_RATE
        self.ALPHA = ALPHA
        
    def __str__(self):
        return str(self.__dict__)
        
config = configurations(IMG_SIZE = (512,512),
                        EPOCHS=8,
                        BATCH_SIZE = 32,
                        DROPOUT_RATE = 0.4,
                        INIT_LEARNING_RATE = 1e-4,
                        ALPHA=0.3)

print(config)

{'IMG_SIZE': (512, 512), 'INPUT_SHAPE': (512, 512, 3), 'EPOCHS': 8, 'BATCH_SIZE': 32, 'DROPOUT_RATE': 0.4, 'INIT_LEARNING_RATE': 0.0001, 'ALPHA': 0.3}


In [8]:
# BASE_DIR = "/kaggle/input/cassava-leaf-disease-classification/"
BASE_DIR = "./"

In [11]:
with open(os.path.join(BASE_DIR, "label_num_to_disease_map.json")) as file:
    map_classes = json.loads(file.read())
    map_classes = {int(k) : v for k, v in map_classes.items()}
    
map_classes

{0: 'Cassava Bacterial Blight (CBB)',
 1: 'Cassava Brown Streak Disease (CBSD)',
 2: 'Cassava Green Mottle (CGM)',
 3: 'Cassava Mosaic Disease (CMD)',
 4: 'Healthy'}

In [13]:
image_file_to_label_df = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))

print(image_file_to_label_df.shape)
print("-"*50)
image_file_to_label_df.head()

(21397, 2)
--------------------------------------------------


,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [19]:
# split into train test sets
X_train, X_val, y_train, y_val = train_test_split(list(image_file_to_label_df.image_id),
                                                  list(image_file_to_label_df.label),
                                                  test_size=0.2)

In [20]:
# appending the base directory to all images' filename
X_train = [os.path.join(os.path.join(BASE_DIR, "train_images"), filename) for filename in X_train]
X_val = [os.path.join(os.path.join(BASE_DIR, "train_images"), filename) for filename in X_val]

In [21]:
# no. of training and validation examples
(len(X_train), len(y_train)), (len(X_val), len(y_val))

((17117, 17117), (4280, 4280))

In [22]:
# turn the labels to categorical
y_train = tf.one_hot(indices=y_train, depth=5)
y_val = tf.one_hot(indices=y_val, depth=5)

In [79]:
def parse_function(filename, label, image_size):
    '''
    Processes the image stored at `filename`. Decode and convert the image to float type, resize it.
    
    :param filename: filepath to the image
    :param label: the label of the image, should be a one-hot vector of length 5
    :param image_size: the desired image size, should be an iterable with 2 elements
    :return resized_image: resized version of the image from the `filename`
    :return label: label of the image
    '''
    image_string = tf.io.read_file(filename)

    #Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=3)

    #This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    resized_image = tf.image.resize(image, image_size)
    
    return resized_image, label

def data_augmentation(image, label):
    '''
    Data augmentation an image for training
    
    :param image: an image, should be an output from parse_function(...)
    :param label: the label of image
    '''
    
    # flips the image laterally at random
    image = tf.image.random_flip_left_right(image)
    
    # randomly adjusts the brightness of the image
    image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
    
    # randomly adjusts the saturation of the image
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)

    #Make sure all the elements in the image is still in [0, 1]
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, label

In [25]:
# process dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(len(X_train))
train_dataset = train_dataset.map(parse_function, num_parallel_calls=4)
train_dataset = train_dataset.map(data_augmentation, num_parallel_calls=4)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(1)

In [26]:
# process dataset for validation
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.shuffle(len(X_val))
val_dataset = val_dataset.map(parse_function, num_parallel_calls=4)
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.prefetch(1)

In [28]:
# weight_filepath = '../input/tfkerasefficientnetimagenetnotop/efficientnetb4_notop.h5'
weight_filepath = '../Model weights/efficient_net/efficientnetb4_notop.h5'

base_model = efficientnet.EfficientNetB4(weights=weight_filepath, 
                                         include_top=False,
                                         input_shape=config.INPUT_SHAPE)

In [84]:
model = Sequential(name="cassava-leaf-disease-net")
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(rate=0.2))
model.add(Dense(5, activation="softmax"))

base_model.trainable = False

In [85]:
model.summary()

Model: "cassava-leaf-disease-net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb4 (Functional)  (None, 16, 16, 1792)      17673823  
_________________________________________________________________
global_average_pooling2d_10  (None, 1792)              0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 1792)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 8965      
Total params: 17,682,788
Trainable params: 8,965
Non-trainable params: 17,673,823
_________________________________________________________________


In [31]:
def label_smoothing(y_true,y_pred):
     return tf.keras.losses.categorical_crossentropy(y_true,y_pred,label_smoothing=0.1)

In [93]:
DECAY_STEPS = int(round(len(X_train)/config.BATCH_SIZE))*config.EPOCHS
cosine_decay = CosineDecay(initial_learning_rate=config.INIT_LEARNING_RATE, decay_steps=DECAY_STEPS, alpha=config.ALPHA)

In [32]:
# compile the model
model.compile(optimizer = Adam(cosine_decay),
              loss = label_smoothing,
              metrics = ['accuracy'])

The below will take very long to run, run with caution.

In [ ]:
history = model.fit(train_dataset,
                    epochs=20,
                    verbose=1,
                    validation_data=val_dataset
                   )

In [40]:
dataset = tf.data.Dataset.range(1, 6)  # ==> [ 1, 2, 3, 4, 5 ]

def random_func(x):
    return [x, x*100]

dataset = dataset.map(random_func)
list(dataset.as_numpy_iterator())

[(1, 100), (2, 200), (3, 300), (4, 400), (5, 500)]